In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import torch
from ultralytics import YOLO
import os

app = Flask(__name__)

# Load the pre-trained YOLO model
model = YOLO('best.pt')

# Define classes
classes = ['mitosis', 'normal']

@app.route('/predict', methods=['GET'])
def predict():
    # Get the image filename from the request query parameter
    filename = request.args.get('image_filename', '')
    
    # Validate the image_filename
    if not filename:
        return jsonify({'error': 'image_filename parameter is missing.'}), 400
    
    # Read the image file
    base_path = r"storage\app\public\images"
    
    image_path = os.path.join(base_path, filename)

    image_path = image_path.replace('\\', '/')
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        return jsonify({'error': f'Error reading the image file: {image_path}'}), 400
    
    # Perform prediction
    with torch.no_grad():
        outputs = model(image)

    # Get prediction result
    prediction = classes[outputs[0].probs.data.argmax()]
    print("Predicted class:", prediction)
    
    # Return prediction result
    return jsonify({'prediction': prediction, 'image_filename': filename, 'outputs':outputs[0].probs.data.tolist()})

if __name__ == '__main__':
    try:
        app.debug = True
        app.run(host='0.0.0.0', use_reloader=False, port=5002)
    except Exception as e:
        print(f"Exception Occurred: {e}")